In [1]:
import pickle
import numpy as np

import pandas as pd
import numpy as np

import mysql.connector as sql
from mysql.connector import Error

import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator

In [ ]:
connection =  sql.connect(
 host="localhost",
 user ="karan",
 password= "Karan@1414",
 database = "database"
 )
print(connection)
cursor = connection.cursor(prepared=True)

# Analysis of the model made :

### Defining the function :

In [2]:
# Define a custom ensemble model that clusters the data based on the mm column and trains an XGBoost model for each cluster

class MmEnsembleModel(BaseEstimator):
    def __init__(self):
        self.models = {}

    def fit(self, X, y):
        # Train an XGBoost model for each mm
        for mm in X['mm'].unique():
            # Filter the training data for the current mm
            X_train_mm = X.loc[X['mm'] == mm, ['year', 'listing_mileage']]

            y_train_mm = y[X['mm'] == mm]

            # Create and train the XGBoost model
            model = XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100)
            model.fit(X_train_mm, y_train_mm)

            # Store the trained model in the dictionary
            self.models[mm] = model

        return self

    
    def predict(self, X):
        # Make predictions using the trained models
        y_pred = []
        for i, row in X.iterrows():
            # Get the mm for the current row
            mm = row['mm']
            
            # Use the corresponding model to make a prediction
            pred = self.models[mm].predict(row[['year', 'listing_mileage']].values.reshape(1, -1))

            
            # Append the prediction to the list of predictions
            y_pred.append(pred[0])
        
        return y_pred
    


### Importing the models:

model_non_rare -> vehicles having count >= 100 in the dataset

model_rare -> vehicles having count < 100 in the dataset

In [3]:
model_non_rare = pickle.load(open('pipe_non_rare.pkl','rb'))
model_rare = pickle.load(open('pipe_rare.pkl','rb'))

### visualising data for hyundai elantra :

In [ ]:
df['cluster'].nunique()

In [ ]:
dt = pd.read_csv('data2.csv')
dt = dt.drop('Unnamed: 0',axis=1)
dt[dt['mm']=='Hyundai Elantra']

In [ ]:
dt = dt[dt['cluster']==1194]

In [ ]:
dt.head(200)

In [ ]:
gh = dt[['year','listing_mileage','listing_price','mm']]

In [ ]:
x= gh.drop('listing_price',axis = 1)

In [ ]:
y = gh['listing_price']

In [ ]:
y_pred = model_non_rare.predict(x)

In [ ]:
xp = x['listing_mileage']

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))
ax.plot(xp, y, linestyle='', marker='o', markersize=10, markeredgewidth=1, markerfacecolor='none', markeredgecolor='olivedrab', label='y',alpha = 0.2)
ax.plot(xp, y_pred, linestyle='', marker='o', markersize=10, markeredgewidth=1.5, markerfacecolor='none', markeredgecolor='brown', label='y_pred',alpha = 0.2)

# Add axis labels and a title
plt.xlabel('x')
plt.ylabel('y')
plt.title('y and y_pred vs x')

plt.legend()
plt.show()

In [ ]:

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
r2 = r2_score(y, y_pred)
r2

### according to year, mm getting cluster:

In [4]:
year = [2020]
mm = ['Toyota Camry']
mileage = [65273]

# Create a dictionary with the column names as keys and the variable values as values
data = {'year': year, 'listing_mileage': mileage, 'mm': mm}

# Create a DataFrame from the dictionary
dft = pd.DataFrame(data)

In [5]:
mm_value = mm[0]
year_value = year[0]

### query to get the cluster of mm

In [6]:
    # Read data from CSV file
    df = pd.read_csv('data3.csv')

    # Filter rows with the specified mm value
    df = df[df['mm'] == mm_value]

    # Get the cluster value
    cluster_value = df['cluster'].iloc[0]

    print(f"The cluster value for '{mm_value}' is: {cluster_value}")


The cluster value for 'Toyota Camry' is: 2383


#### predicting listing price:

In [7]:
if cluster_value == 2618:
    
    prd_price = model_rare.predict(dft)
else:
    prd_price = model_non_rare.predict(dft)
    
    


In [8]:
print(prd_price)

[27114.393]


In [9]:
print(type(prd_price))

<class 'list'>


### function to get the top 100 realated data according to our target data predicted

In [10]:
def get_similar_rows(year, mm, cluster_value):
    # Read data from CSV file
    df = pd.read_csv('data3.csv')
    
    n = 100
    if cluster_value == 2618:
        # Filter rows with the specified mm and cluster values
        df = df[(df['mm'] == mm) & (df['cluster'] == cluster_value)]
        return df
    else:
        # Filter rows with the specified mm and years in the specified range
        min_year = 1980
        max_year = 2025
        df = df[(df['mm'] == mm) & (df['year'] >= min_year) & (df['year'] <= max_year)]
        
        # Sort rows by the absolute difference between year and the specified year
        df['year_diff'] = (df['year'] - year).abs()
        df = df.sort_values('year_diff')
        
        # Select the first n rows
        df = df.head(n)
        
        # Select the desired columns
        df = df[['year', 'mm', 'listing_mileage', 'listing_price']]
        
        # Convert columns to the desired data types and formats
        df['listing_mileage'] = df['listing_mileage'].astype(int)
        df['listing_price'] = df['listing_price'].astype(int)
        
        df['listing_mileage'] = df['listing_mileage'].astype(str)
        df['listing_price'] = df['listing_price'].astype(str)
        
        df['listing_mileage'] =  df['listing_mileage'] + ' miles'
        df['listing_price'] = '$' + df['listing_price']
        print(df)
        return df





In [11]:
# Example usage
gh = get_similar_rows(year_value,mm_value,cluster_value)
gh

         year            mm listing_mileage listing_price
3797725  2020  Toyota Camry     61499 miles        $26998
3745077  2020  Toyota Camry     21515 miles        $30053
3851965  2020  Toyota Camry     59959 miles        $27998
3752658  2020  Toyota Camry     34430 miles        $30000
3745087  2020  Toyota Camry      6615 miles        $34989
...       ...           ...             ...           ...
3767136  2020  Toyota Camry     31753 miles        $35999
3860444  2020  Toyota Camry     33908 miles        $27991
3860445  2020  Toyota Camry     28231 miles        $29998
3767149  2020  Toyota Camry     10389 miles        $33470
3745285  2020  Toyota Camry     25079 miles        $32441

[100 rows x 4 columns]


,year,mm,listing_mileage,listing_price
3797725,2020,Toyota Camry,61499 miles,$26998
3745077,2020,Toyota Camry,21515 miles,$30053
3851965,2020,Toyota Camry,59959 miles,$27998
3752658,2020,Toyota Camry,34430 miles,$30000
3745087,2020,Toyota Camry,6615 miles,$34989
...,...,...,...,...
3767136,2020,Toyota Camry,31753 miles,$35999
3860444,2020,Toyota Camry,33908 miles,$27991
3860445,2020,Toyota Camry,28231 miles,$29998
3767149,2020,Toyota Camry,10389 miles,$33470


### Checking for one rare model:

In [ ]:
import mysql.connector
cnx = mysql.connector.connect(
    host="localhost",
    user="karan",
    password="Karan@1414",
    database="database"
)

# Create a query to fetch all rows from the table
query = "SELECT * FROM data2 where cluster = 2618"

# Execute the query and store the result in a DataFrame
df2 = pd.read_sql(query, cnx)

# Close the connection
cnx.close()
df2 = df2.drop('s_no', axis = 1)

In [ ]:
gh = df2
gh

In [ ]:
mm_value = 'Forest River TOYHAULER VISION 31093400'
year_value = 2005

In [ ]:
query = "SELECT cluster FROM data2 WHERE mm = %s"
cursor.execute(query, (mm_value,))

result = cursor.fetchall()

cluster_value = result[0][0]
print(f"The cluster value for'{mm_value}' is: {cluster_value}")


In [ ]:
gh = get_similar_rows(year_value,mm_value,cluster_value)

In [ ]:
data = [['2005','52377','Forest River TOYHAULER VISION 31093400']]
columns = ['year', 'listing_mileage', 'mm']

dt_r = pd.DataFrame(data, columns=columns)


model_rare.predict(dt_r)